# Protocol lab


Here we will walk through the steps of building up a firmware update
protocol. We are going to use Python to implement the updater tool
side of the protocol (the other side being the bootloader, written
in C). This protocol can get fairly complex, so focus on structuring
your code well so that it can be extended a re-used throughout the
challenges!


### Challenge Name: sending_data (/embsec/protocol_lab/sending_data)


Implement the protocol described here, and send the contents of the
sending_data.bin file to the bootloader.

Protocol Description: Messages are of the format specified below,
and are always 64 bytes. Datatypes are all big-endian.

             [short]        [58 bytes]  [short]    [short]
     -------------------------------------------------------
    | message type (short) | data ... | unused_1 | unused_2 |
     -------------------------------------------------------

To begin a firmware update, send a BEGIN message (message type = 1)
to the bootloader, the other fields can be set to anything for this
message.

Next, send the sending_data.bin file contents to the bootloader in chunks. For
this, send FIRMWARE (message type = 2) messages, where the data field
is the partial file contents.

Once you have sent the entire file, send a DONE (type = 3) message and
then listen for a line that will contain the flag.

For simplicity, the firmware file size is always a multiple of 58 bytes.




In [1]:
import embsec
import struct

ser = embsec.Serial('/embsec/protocol_lab/sending_data')
ser.write(struct.pack('>h58shh', 1, bytes(58), 0, 0)) #begin message- type 1, fully populate information fields even if not in use rn
#general for format of this is above

with open('sending_data.bin', 'rb') as fp: #basic
    file_contents = fp.read()
    #stuff in with is accessible outside (in terms of scope)

for i in range(0, len(file_contents), 58): #breaking file into chunks
    #the 58 is the step size, in bytes
    file_chunk = file_contents[i: i+58] #getting a 58 byte piece
    frame = struct.pack('>h58shh', 2, file_chunk, 0, 0)
    ser.write(frame)
    
ser.write(struct.pack('>h58shh', 3, bytes(58), 0, 0))
print(ser.read_until())
#array length not a multiple of 58? not an issue here but is elsewhere
#bytes(58) sends 58 empty bytes

b'embsec{sending_data_e3993c1af17976bb}\n'


### Challenge Name: heartbeat (/embsec/protocol_lab/heartbeat)


The bootloader can only take so much data at once and is getting bogged down by
the constant stream of data sent its way. It cannot write the firmware to memory
as fast as it's receiving new data.

Now, the bootloader will send a OK (message type = 4) message when it is all
set to be sent another messsage. So, before sending any message, wait for an
OK message, and then send just one message. After you have sent a DONE message,
the bootloader will send an OK message, and then a flag.



In [4]:
from embsec import Serial

def heartbeat():
    ser = Serial("/embsec/protocol_lab/heartbeat")
    ser.write(struct.pack('>h58shh', 1, bytes(58), 0, 0)) #begin message- type 1, fully populate information fields even if not in use rn
    #general for format of this is above

    with open('sending_data.bin', 'rb') as fp: #basic
        file_contents = fp.read()
        #stuff in with is accessible outside (in terms of scope)

    for i in range(0, len(file_contents), 58): #breaking file into chunks
        #the 58 is the step size, in bytes
        file_chunk = file_contents[i: i+58] #getting a 58 byte piece
        frame = struct.pack('>h58shh', 2, file_chunk, 0, 0)
        
        got_data = ser.read(64) #messages are always 64 bytes
        got_packet = struct.unpack('>h58shh', got_data) #how to unpack, the data to unpack
        #unpack will return a tuple
        rec_type = got_packet[0] #message type is first thing

        ser.write(frame)

    ser.write(struct.pack('>h58shh', 3, bytes(58), 0, 0))
    
    ser.read(64)
    
    print(ser.read_until())

heartbeat()


b'\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00embsec{heartbeat_f300c70d84c5317b}\n'


### Challenge Name: noise (/embsec/protocol_lab/noise)


Your data is being sent over a noisy medium - some messages are getting corrupted
and the bootloader cannot interpret the message, causing firmware update
failures.

Add a checksum to the end of the message in the unused_1 field. This checksum is
the sum of all the bytes in the data field.

When the bootloader receives any message, it will calculate the checksum. There is a
chance that an error occured during transmission, and this calculated value may
differ from the checksum field of the message. If this is the case, the bootloader
will send a RESEND (message type = 5) message, and expect the last message to be
re-sent.

If you receive a RESEND message instead of a OK message, re-send the last frame.
This includes the DONE message, too!




In [2]:
from embsec import Serial
import struct

def noise():
    ser = Serial("/embsec/protocol_lab/noise")
    
    #Start Message
    ser.write(struct.pack('>h58shh', 1, bytes(58), 0, 0))
    
    # Your code goes here!
    with open('sending_data.bin', 'rb') as fp:
        file_contents = fp.read()
        
    for i in range(0, len(file_contents), 58):
        file_chunk = file_contents[i:i+58]
        checksum = sum(file_chunk)
        print(checksum)
        
        ser.write(struct.pack('>h58shh', 2, file_chunk, checksum, 0))
        while struct.unpack('>h58shh', ser.read(64))[0] == 5:
            print("RESEND")
            ser.write(struct.pack('>h58shh', 2, file_chunk, checksum, 0))
            print(file_chunk)
            
    print("hello")
    
    ser.write(struct.pack('>h58shh', 3, bytes(58), 0, 0))
        
    while struct.unpack('>h58shh', ser.read(64))[0] == 5:
        print("RESEND")
        ser.write(struct.pack('>h58shh', 3, bytes(58), 0, 0))
        
    return ser.read_until()
        
noise()


6282
6282
6282
6282
6282
6282
6282
6282
6282
6282
6282
6282
6282
6282
6282
6282
hello


b'\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00embsec{noise_556e39e67bbc176a}\n'

### Challenge Name: replay (/embsec/protocol_lab/replay)


There have been instances of counterfeit update tools being used to upload
modified firmware to cars. To prevent this, the auto company has tried to
secure their protocol.

In order for the bootloader to verify the update tool is legitimate, a handshake
is added to the beginning of the communication protocol. Now, messages are sent
as follows:

1. The update tool sends a BEGIN message to the bootloader, as before.

2. The bootloader responds with a CHALLENGE message (type = 6) where the
   data field contains a random number.

3. The update tool takes the data from the CHALLENGE message and performs a
   secret mathematical operations on it. The bootloader also knows what this
   operation is, but you do not. The update tool must respond
   with an ANSWER message (type = 7) where the data field is the result of the
   secret mathematical operation on the challenge data.

4. If the ANSWER is correct, the communications proceed normally. If incorrect, the
   bootloader will respond WRONG (type = 8), and not accept any more information for that session.
   You will have to wait 1 second before starting a new session.

You are an attacker who has a counterfeit update tool. You need to figure out a way
to make it through this handshake and send modified firmware. You have attached a
protocol analyzer to the bus, and can read out all communications. Doing this, you
have noticed a crucial bit of information: the CHALLENGE data is random, but appears
to only be values between [0, 255].

1. How can you get past this handshake? (HINT: look at the listed resources!)

2. Can you get away with observing just one legitimate handshake? How so?

3. What is the expected maximum time the attack in question two can take? Remember that
   penalty for a failed handshake is 1 second, ignore the time the actual
   handshake takes for now.

4. BONUS: implement your attack here, in python. To listen to traffic on the bus,
   use ser.read(64), and you don't have to worry about checksums for this challenge.
   Once you have enough information and have waited for a handshake to complete, use
   ser.write() to mount your attack. Don't worry about the 1 second delay for this
   attack.

Resources:

<https://en.wikipedia.org/wiki/Replay_attack>




In [ ]:
from embsec import Serial
import struct


def replay(): 
    ser = Serial("/embsec/protocol_lab/replay")
    #start message
    #ser.write(struct.pack('>h58shh', 1, bytes(58), 0, 0)) #the 1 is the message type 
    #the 0's represent unused stuff
    b = ser.read(64)
    c = ser.read(64)
    a = ser.read(64)
    d = ser.read(64)
    #basically I need to get the answer and write that
    challenge = struct.unpack('>h58shh', c)
    rand = challenge[1]
    
    while (True):
        ser.write(struct.pack('>h58shh', 1, bytes(58), 0, 0))
        chal = ser.read(64)
        challenge2 = struct.unpack('>h58shh', c)
        rand2 = challenge2[1]
        if(rand == rand2):
            #ser.write(struct.pack('>h58shh', 1, rand, 0, 0))
            ser.write(a)
            break
        else:
            ser.write(struct.pack(">h58shh", 7, bytes(58), 0, 0))
            ser.read(64)
    
    return ser.read_until()

replay()